In [167]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots

In [196]:
using SparseArrays

In [210]:
#load the data and orgnize 
cap_cstr = CSV.read("/Users/bourg/.julia/environments/batterySC/data/testdata/battery/capacity_constraints18.csv", 
                    DataFrame, 
                    header=1) |> DataFrame

distance = CSV.File("/Users/bourg/.julia/environments/batterySC/data/testdata/battery/distance18.csv",header=1) |> DataFrame

LCA_model = CSV.File("/Users/bourg/.julia/environments/batterySC/data/testdata/battery/LCA_model18.csv",header=1) |> DataFrame

SD = CSV.File("/Users/bourg/.julia/environments/batterySC/data/testdata/battery/emission_sink18.csv",header=1) |> DataFrame; 

# first(cap_cstr, 5)
# first(distance, 5)
# ncol(distance)

In [211]:
first(cap_cstr,10)

Row,Country,NMC111 powder,Graphite,PP,PE,Cu,Al,battery production
,String31,Float64,Int64,Int64,Int64,Int64,Int64,Float64
1,Argentina,0.0,0,300000,0,0,0,0.0
2,Australia,0.0,500,500000,600000,920000,1600000,0.0
3,Bahrain,0.0,0,0,0,0,1200000,0.0
4,Bolivia,0.0,0,0,0,0,0,0.0
5,Brazil,0.0,87000,1900000,3600000,400000,590000,0.0
6,Canada,0.0,15000,1400000,2500000,480000,2900000,0.0
7,Chile,0.0,0,0,300000,5700000,0,0.0
8,China,1.0e11,850000,23800000,27000000,1600000,37000000,1.0e15
9,Colombia,0.0,0,200000,600000,0,0,0.0


In [212]:
LCA_model

Row,process,EF (kg CO2/material),input (material/kg battery)
,String31,Float64,Float64
1,NMC111 powder,14.7,0.252
2,Graphite,4.86,0.141
3,PP,2.53,0.011
4,PE,2.93,0.00364
5,Cu,3.08,0.117
6,Al,7.41,0.239
7,battery production,0.1183,1.0


In [213]:
# powder_demand = 7000                        # demand of NMC111 powder
battery_demand = 500000
gobal_sink = 1.53e9                         # global CO2 sequestration
global_emission = 4.75e10                   # global CO2 emission
EF_aircraft = 0.000433                     # kg CO2/km*ton emission factor for freight transporation 
EF_input = LCA_model[!,"EF (kg CO2/material)"]
process = LCA_model[!,"process"]
countries = cap_cstr[!,"Country"]
n = size(countries,1)                       # No. of countries
m = size(process,1);                        # No. of processes 

In [228]:
# seperate model
scaler_battery = LCA_model[1:7,"input (material/kg battery)"]     # mineral inputs for 1 kg NMC111 powder

7-element Vector{Float64}:
 0.252
 0.141
 0.011
 0.00364
 0.117
 0.239
 1.0

In [229]:
input_FU = battery_demand .* scaler_battery

7-element Vector{Float64}:
 126000.0
  70500.0
   5500.0
   1820.0
  58500.0
 119500.0
 500000.0

----

In [222]:
model = Model(Gurobi.Optimizer);

Academic license - for non-commercial use only - expires 2023-11-27


In [223]:
#variables 
@variable(model, x[1:n, 1:m] >= 0)         # x[i,k] production amount of product k at location i
@variable(model, y[1:n, 1:n, 1:m] >= 0);   # y[i,j,k] ship product k from i to j 

In [224]:
# node output flow constraint
for k in 1:m
    for i in 1:n
        @constraint(model, sum(y[i,j,k] for j in 1:n) <= x[i,k])
    end
end

In [225]:
# NMC111 powder input flow constraint
for k in 1:4
    for j in 1:n
        @constraint(model, sum(y[i,j,k] for i in 1:n) >= x[j,5] * scaler_battery[k])
#         @constraint(model, sum(y[i,j,k] for i in 1:n) >= input_FU[k])
    end
end

In [226]:
# production capacity constraints
for i in 1:n
    for k in 1:m
        @constraint(model, x[i,k] <= cap_cstr[!, 2:end][i,k])
    end
end

In [230]:
input_FU

7-element Vector{Float64}:
 126000.0
  70500.0
   5500.0
   1820.0
  58500.0
 119500.0
 500000.0

In [231]:
# final demand constraints
for k in 1:6
    @constraint(model, sum(x[i,k] for i in 1:n) == input_FU[k])
end

@constraint(model, sum(x[i,7] for i in 1:n) == battery_demand)

x[1,7] + x[2,7] + x[3,7] + x[4,7] + x[5,7] + x[6,7] + x[7,7] + x[8,7] + x[9,7] + x[10,7] + x[11,7] + x[12,7] + x[13,7] + x[14,7] + x[15,7] + x[16,7] + x[17,7] + x[18,7] + x[19,7] + x[20,7] + x[21,7] + x[22,7] + x[23,7] + x[24,7] + x[25,7] + x[26,7] + x[27,7] + x[28,7] + x[29,7] + x[30,7] + x[31,7] + x[32,7] + x[33,7] + x[34,7] + x[35,7] + x[36,7] + x[37,7] + x[38,7] + x[39,7] + x[40,7] == 500000.0

In [234]:
vec_flow = []

for k in 1:m
    flow = sum(x[i, k] for i in 1:n)
    push!(vec_flow, flow)
end

D_proc = EF_input' * vec_flow;

In [235]:
trans_emi = 0

for k in 1:m
    for i in 1:n
        for j in 1:n
            trans_emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_aircraft
        end
    end
end

# output = sum(x[i,5] for i in 1:n)
D_trans = trans_emi + sum(x[i,7] * distance[!, 2:end][i,38] for i in 1:n);

┌ Warning: The addition operator has been used on JuMP expressions a large number of times. This warning is safe to ignore but may indicate that model generation is slower than necessary. For performance reasons, you should not add expressions in a loop. Instead of x += y, use add_to_expression!(x,y) to modify x in place. If y is a single variable, you may also use add_to_expression!(x, coef, y) for x += coef*y.
└ @ JuMP C:\Users\bourg\.julia\packages\JuMP\klrjG\src\JuMP.jl:1352


In [185]:
# SD[!, "sink ton/yr"]

In [186]:
# EF_input

In [236]:
# supply - based on emission
emi = x * EF_input
S_country = SD[!, "sink ton/yr"] .* emi ./ SD[!, "emission"]
S_global = gobal_sink/global_emission * emi
S = S_country + S_global
S_tot = sum(S);

In [237]:
@objective(model, Min, D_proc + D_trans);

In [238]:
JuMP.optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 727 rows, 11480 columns and 18600 nonzeros
Model fingerprint: 0xe1e2e9a5
Coefficient statistics:
  Matrix range     [4e-03, 1e+00]
  Objective range  [1e-01, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+15]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 627 rows and 10813 columns
Presolve time: 0.01s
Presolved: 100 rows, 667 columns, 1373 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.3389026e+06   3.698087e+04   0.000000e+00      0s
      22    3.3389026e+06   0.000000e+00   0.000000e+00      0s

Solved in 22 iterations and 0.01 seconds
Optimal objective  3.338902600e+06

User-callback calls 65, time in user-callback 0.00 sec


In [240]:
optY = JuMP.value.(y);

In [241]:
optX = JuMP.value.(x)

40×7 Matrix{Float64}:
      0.0      0.0     0.0     0.0       0.0       0.0       0.0
      0.0    500.0     0.0     0.0       0.0       0.0       0.0
      0.0      0.0     0.0     0.0       0.0       0.0       0.0
      0.0      0.0     0.0     0.0       0.0       0.0       0.0
      0.0  29851.5  4856.5     0.0       0.0       0.0       0.0
      0.0  15000.0     0.0     0.0       0.0       0.0       0.0
      0.0      0.0     0.0     0.0       0.0       0.0       0.0
 126000.0   8248.5   643.5   212.94  58500.0       0.0       0.0
      0.0      0.0     0.0  1607.06      0.0       0.0       0.0
      0.0      0.0     0.0     0.0       0.0       0.0       0.0
      0.0      0.0     0.0     0.0       0.0       0.0       0.0
      0.0      0.0     0.0     0.0       0.0       0.0       0.0
      0.0      0.0     0.0     0.0       0.0       0.0       0.0
      ⋮                                            ⋮    
      0.0      0.0     0.0     0.0       0.0       0.0       0.0
      0.0  

In [23]:
input_FU

4-element Vector{Float64}:
  503.99999999999994
 1421.0
 1330.0
 1421.0

In [191]:
JuMP.value.(y[1:10,1:10,1])

10×10 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  504.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0

In [25]:
JuMP.value.(y[1:10,1:10,2])

10×10 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  931.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  490.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0

In [26]:
JuMP.value.(y[1:10,1:10,3])

10×10 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1330.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0

In [27]:
JuMP.value.(y[1:10,1:10,4])

10×10 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1421.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0

In [28]:
JuMP.value.(y[1:10,1:10,5])

10×10 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [29]:
distance

Row,Column1,Argentina,Australia,Brazil,Canada,Chile,China,Russia,United States,Zambia,Zimbabwe
,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Argentina,0.0,12704.8,2924.25,11259.1,766.299,18899.0,17335.1,9017.69,9230.47,8940.93
2,Australia,12704.8,0.0,15578.6,14151.6,12734.1,7474.36,9983.62,15184.5,10932.2,10496.6
3,Brazil,2924.25,15578.6,0.0,9304.54,3084.2,16632.9,14451.5,7316.93,8571.87,8579.94
4,Canada,11259.1,14151.6,9304.54,0.0,10725.8,9385.87,6644.49,2261.54,13848.5,14485.1
5,Chile,766.299,12734.1,3084.2,10725.8,0.0,19630.1,17132.4,8466.86,9985.79,9705.13
6,China,18899.0,7474.36,16632.9,9385.87,19630.1,0.0,2854.62,11647.3,9668.54,9963.54
7,Russia,17335.1,9983.62,14451.5,6644.49,17132.4,2854.62,0.0,8886.1,10639.3,11151.6
8,United States,9017.69,15184.5,7316.93,2261.54,8466.86,11647.3,8886.1,0.0,13844.5,14330.4
9,Zambia,9230.47,10932.2,8571.87,13848.5,9985.79,9668.54,10639.3,13844.5,0.0,668.693


### TEST

In [100]:
# vec_flow = []

# for k in 1:5
#     flow = sum(optX[i, k] for i in 1:10)
#     push!(vec_flow, flow)
# end

# D_proc = EF_input' * vec_flow

In [56]:
function proc_emi(x, n, m, EF_input)
    vec_flow = []
    for k in 1:m
        flow = sum(x[i,k] for i in 1:n)
        push!(vec_flow, flow)
    end
    return EF_input' * vec_flow
end

proc_emi (generic function with 1 method)

In [61]:
function transp_emi(y,n,m)
    ls = []
    for k in 1:m
        for i in 1:n
            for j in 1:n
                emission = y[i,j,k] * distance[!, 2:end][i,j] * EF_aircraft
                if emission > 0
                    push!(ls, emission)
                end
            end
        end
    end
    return sum(ls)
end

transp_emi (generic function with 1 method)

In [93]:
function carbon_supply(x,EF_input,SD,gobal_sink,global_emission)
    emi = x * EF_input
    S_country = SD[!, "sink ton/yr"] .* emi ./ SD[!, "emission"]
    S_global = gobal_sink/global_emission * emi
    S = S_country + S_global
    return sum(S)
end

carbon_supply (generic function with 1 method)

In [104]:
feaY = copy(optY)
# typeof(feaY)
feaY[4,8,2] = 0
feaY[2,8,2] += 931;

In [106]:
feaY[1:10,1:10,2]

10×10 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  931.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  490.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0

In [110]:
feaX = copy(optX)
feaX[4,2] = 0
feaX[2,2] = 931
feaX

10×5 Matrix{Float64}:
   0.0    0.0     0.0     0.0     0.0
   0.0  931.0     0.0     0.0     0.0
   0.0    0.0  1330.0     0.0     0.0
   0.0    0.0     0.0  1421.0     0.0
   0.0    0.0     0.0     0.0     0.0
   0.0    0.0     0.0     0.0     0.0
   0.0    0.0     0.0     0.0     0.0
 504.0  490.0     0.0     0.0  7000.0
   0.0    0.0     0.0     0.0     0.0
   0.0    0.0     0.0     0.0     0.0

In [111]:
trans_test = transp_emi(feaY,10,5)

1.1726478011691008e7

In [112]:
proc_test = proc_emi(feaX, 10, 5, EF_input)

112575.414

In [113]:
Stest = carbon_supply(feaX,EF_input,SD,gobal_sink,global_emission)

58169.53425798187

In [114]:
result = trans_test+proc_test-Stest

1.1780883891433027e7

In [107]:
# m = 5
# n = 10
# ls = []

# for k in 1:m
#     for i in 1:n
#         for j in 1:n
#             trans_emi = optY[i,j,k] * distance[!, 2:end][i,j] * EF_aircraft
#             if trans_emi > 0
#                 push!(ls, trans_emi)
#             end
#         end
#     end
# end

In [35]:
ls

3-element Vector{Any}:
 911678.2625382389
      4.213746169823749e6
      1.3915089270320488e6

In [42]:
D_trans = sum(ls)

6.516933359394037e6

In [37]:
optY[4,8,2] * distance[!, 2:end][4,8] * EF_aircraft

911678.2625382389

In [38]:
optY[3,8,3] * distance[!, 2:end][3,8] * EF_aircraft

4.213746169823749e6

In [43]:
D_proc + D_trans

6.629508773394037e6

In [47]:
emi = optX * EF_input
S_country = SD[!, "sink ton/yr"] .* emi ./ SD[!, "emission"]

10-element Vector{Float64}:
     0.0
     0.0
 16460.170940170938
  4375.268162601626
     0.0
     0.0
     0.0
 34382.964
     0.0
     0.0

In [48]:
S_global = gobal_sink/global_emission * emi

10-element Vector{Float64}:
    0.0
    0.0
  239.90399999999997
   63.729234947368425
    0.0
    0.0
    0.0
 3322.4801002105264
    0.0
    0.0

In [50]:
S = S_country + S_global 
S_tot = sum(S)

58844.516437930455

10×5 Matrix{Float64}:
   0.0     0.0     0.0     0.0     0.0
   0.0     0.0     0.0     0.0     0.0
   0.0     0.0  1330.0     0.0     0.0
   0.0     0.0     0.0  1421.0     0.0
   0.0     0.0     0.0     0.0     0.0
   0.0     0.0     0.0     0.0     0.0
   0.0     0.0     0.0     0.0     0.0
 504.0  1421.0     0.0     0.0  7000.0
   0.0     0.0     0.0     0.0     0.0
   0.0     0.0     0.0     0.0     0.0